# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at mlta-2022-spring@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1H5ZONrb2LMOCixLY7D5_5-7LkIaXO6AGEaV2mRdTOMY/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/c/ml2022spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2.5hrs
  

## Download Dataset

In [1]:
# import gdown
# # Download link 1
# !gdown --id '1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb' --output hw7_data.zip

# # Download Link 2 (if the above link fails) 
# # !gdown --id '1qwjbRjq481lHsnTrrF4OjKQnxzgoLEFR' --output hw7_data.zip

# # Download Link 3 (if the above link fails) 
# # !gdown --id '1QXuWjNRZH6DscSd6QcRER0cnxmpZvijn' --output hw7_data.zip

# !unzip -o hw7_data.zip

# # For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
# !nvidia-smi

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [2]:
# # You are allowed to change version of transformers or use other toolkits
# !pip install transformers==4.18.0

## Import Packages

In [3]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast, get_linear_schedule_with_warmup
import os

from tqdm.auto import tqdm

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(777777)

In [4]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    # !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

## Load Model and Tokenizer




 

In [5]:
# model = BertForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")
pre_model = "luhua/chinese_pretrain_mrc_macbert_large"
tokenizer = BertTokenizerFast.from_pretrained(pre_model)
model = BertForQuestionAnswering.from_pretrained(pre_model).to(device)

# tokenizer = RobertaTokenizer.from_pretrained('hfl/chinese-roberta-wwm-ext')
# model = RobertaModel.from_pretrained('hfl/chinese-roberta-wwm-ext')

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

## Read Data

- Training set: 31690 QA pairs
- Dev set: 4131  QA pairs
- Test set: 4957  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [6]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [7]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [8]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 50
        self.max_paragraph_len = 350
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 310

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn
        exceed = True if len(tokenized_paragraph) > self.max_paragraph_len else False

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            if exceed:
                mid = (answer_start_token + answer_end_token) // 2
                paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            else:
                rand_start = random.randint(0, answer_start_token)
                paragraph_start = rand_start
            paragraph_end = paragraph_start + self.max_paragraph_len

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            qa_offset_list = []
            p_offset_list = []
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # calculate qa offset
                qa_offset = len(input_ids_question)
                qa_offset_list.append(qa_offset)
                p_offset = tokenized_paragraph.offsets[i : i + self.max_paragraph_len]
                p_offset_list.append(p_offset)
                # print(f"qa: {len(input_ids_question)})")
                # print(f"para: {len(input_ids_paragraph)})")
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list), torch.tensor(qa_offset_list), p_offset_list

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 4

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Training

In [9]:
num_epoch = 3
validation = False
logging_step = 100
learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)

# batch accumulation parameter
accum_iter = 2

#set up scheduler
len_dataset = len(train_set)
print(f"length of train_set: {len_dataset}")
total_steps = (len_dataset // accum_iter) * num_epoch
warm_up_ratio = 0
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = warm_up_ratio * total_steps, num_training_steps = total_steps)



if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

model.train()

print("Start Training ...")

best_acc = 0

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()

        # normalize loss to account for batch accumulation
        train_loss += output.loss / accum_iter 
        
        if fp16_training:
            accelerator.backward(output.loss)
        else:
            output.loss.backward()
        
        # weights update
        if (step % accum_iter == 0) or (step == len(train_loader)):
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            
        # optimizer.step()
        # optimizer.zero_grad()
        step += 1

        # # ##### TODO: Apply linear learning rate decay #####
        # scheduler.step()
        
        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0



    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")

            if dev_acc > best_acc:
                best_acc = dev_acc
                print("Saving Model ...")
                model_save_dir = "saved_model_last" 
                model.save_pretrained(model_save_dir)
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」

print("Saving Model ...")
model_save_dir = "saved_model_last" 
model.save_pretrained(model_save_dir)

/home/tracy/miniconda3/envs/torch11/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


length of train_set: 31690
Start Training ...


  0%|          | 1/7923 [00:00<1:28:30,  1.49it/s]/home/tracy/miniconda3/envs/torch11/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
  1%|▏         | 100/7923 [00:14<19:04,  6.84it/s]

Epoch 1 | Step 100 | loss = 0.496, acc = 0.625


  3%|▎         | 200/7923 [00:27<19:05,  6.74it/s]

...

Epoch 3 | Step 7800 | loss = 0.036, acc = 0.967


100%|█████████▉| 7900/7923 [18:39<00:03,  6.56it/s]

Epoch 3 | Step 7900 | loss = 0.063, acc = 0.927


100%|██████████| 7923/7923 [18:42<00:00,  7.06it/s]


Saving Model ...


In [10]:
max_answer_length = 30

def evaluate(data, output1, paragraph):

    p_offsets = data[4]
    question_offset = data[3].squeeze(dim=0)
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    n_best = 60

    
    ans_start, ans_end, ans_k  = 0,0,0
    for k in range(num_of_windows):
        output_start = output1.start_logits[k].cpu().numpy()
        output_end = output1.end_logits[k].cpu().numpy()

        start_indexes = np.argsort(output_start)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(output_end)[-1 : -n_best - 1 : -1].tolist()

        for start_index in start_indexes:
            for end_index in end_indexes:
                if start_index > end_index or end_index - start_index + 1 > max_answer_length:
                    continue
                
                if start_index < question_offset[k].item():
                    # print(f"answer is in question: {start_index},{end_index}")
                    continue
                
                start_prob= output1.start_logits[k][start_index]
                end_prob = output1.end_logits[k][end_index]
                
                prob = start_prob + end_prob
                if prob > max_prob:
                    max_prob = prob
                    # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
                    answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
                    ans_start, ans_end, ans_k = start_index , end_index, k


    # print(ans_start, ans_end, ans_k)
    # print(paragraph)
    
    res = ''
    prefix = question_offset[ans_k].item() # char occupied by question

    ans_p_start = ans_start  -  prefix #ans_start in origin paragraph token
    ans_p_end = ans_end -  prefix


    # tks = ids[ans_k][ans_start:ans_end+1].tolist()
    # print(tks)
    st, ed = 0,0
    p_offset = [(a.item(),b.item()) for a,b in p_offsets[ans_k]]
    # print(len(p_offset), ans_p_start, ans_p_end, ans_k)
    for i in range(ans_p_start, ans_p_end+1):
        st, ed = p_offset[i]
        res += paragraph[st : ed]
    # print(f"k: {ans_k}, res: {res}")
    # raise Exception
    # if '[UNK]' in answer:
    #     print('found [UNK] in prediction, using original text')
    #     print('original prediction', answer)
    #     print('final prediction',res)
        
    # res = res.replace(' ','')
    # print(res)
    return res

def ensemble(data, output1, output2, output5, paragraph):

    p_offsets = data[4]
    question_offset = data[3].squeeze(dim=0)
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    n_best = 60

    
    ans_start, ans_end, ans_k  = 0,0,0
    for k in range(num_of_windows):
        # output_start = output1.start_logits[k].cpu().numpy() + output2.start_logits[k].cpu().numpy() + output5.start_logits[k].cpu().numpy()
        # output_end = output1.end_logits[k].cpu().numpy() + output2.end_logits[k].cpu().numpy() + output5.end_logits[k].cpu().numpy()
        output_start = output1.start_logits[k].cpu().numpy() + output2.start_logits[k].cpu().numpy()
        output_end = output1.end_logits[k].cpu().numpy() + output2.end_logits[k].cpu().numpy()

        start_indexes = np.argsort(output_start)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(output_end)[-1 : -n_best - 1 : -1].tolist()

        for i, start_index in enumerate(start_indexes):
            for j, end_index in enumerate(end_indexes):
                if start_index > end_index or end_index - start_index + 1 > max_answer_length:
                    continue

                if start_index < question_offset[k].item():
                    # print(f"answer is in question: {start_index},{end_index}")
                    continue
                
                # start_prob= output1.start_logits[k][start_index] + output2.start_logits[k][start_index] + output5.start_logits[k][start_index]
                # end_prob = output1.end_logits[k][end_index]+ output2.end_logits[k][end_index] + output5.end_logits[k][end_index]
                start_prob= output1.start_logits[k][start_index] + output2.start_logits[k][start_index]
                end_prob = output1.end_logits[k][end_index]+ output2.end_logits[k][end_index]
                
                prob = start_prob + end_prob
                if prob > max_prob:
                    max_prob = prob
                    # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
                    answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
                    ans_start, ans_end, ans_k = start_index , end_index, k


    # print(ans_start, ans_end, ans_k)
    # print(paragraph)
    
    res = ''
    prefix = question_offset[ans_k].item() # char occupied by question

    ans_p_start = ans_start  -  prefix #ans_start in origin paragraph token
    ans_p_end = ans_end -  prefix


    # tks = ids[ans_k][ans_start:ans_end+1].tolist()
    # print(tks)
    st, ed = 0,0
    p_offset = [(a.item(),b.item()) for a,b in p_offsets[ans_k]]
    # print(p_offset)
    for i in range(ans_p_start, ans_p_end+1):
        st, ed = p_offset[i]
        res += paragraph[st : ed]
    # print(f"k: {ans_k}, res: {res}")
    # raise Exception
    # if '[UNK]' in answer:
    #     print('found [UNK] in prediction, using original text')
    #     print('original prediction', answer)
    #     print('final prediction',res)
        
    # res = res.replace(' ','')
    # print(res)
    return res

In [11]:
# model2 = BertForQuestionAnswering.from_pretrained("saved_model_cml5_2").to(device)

In [12]:
# print("Evaluating Dev Set ...")
# model.eval()
# with torch.no_grad():
#     dev_acc = 0
#     for i, data in enumerate(tqdm(dev_loader)):
#         output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
#                 attention_mask=data[2].squeeze(dim=0).to(device))
#         output2 = model2(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
#                 attention_mask=data[2].squeeze(dim=0).to(device))
#         # prediction is correct only if answer text exactly matches
#         dev_acc += ensemble(data, output,output2, None, dev_paragraphs[dev_questions[i]['paragraph_id']]) == dev_questions[i]["answer_text"]
#     print(f"Validation | acc = {dev_acc / len(dev_loader):.3f}")

In [13]:
# result = []

# with torch.no_grad():
#     for i, data in enumerate(tqdm(test_loader)):
#         output1 = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
#                        attention_mask=data[2].squeeze(dim=0).to(device))
        
#         output2 = model2(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
#                        attention_mask=data[2].squeeze(dim=0).to(device))

#         output5 = None
        
#         result.append(ensemble(data, output1, output2, output5, test_paragraphs[test_questions[i]['paragraph_id']]))


# result_file = "result_ensemble_last1_6.csv"
# with open(result_file, 'w') as f:	
# 	  f.write("ID,Answer\n")
# 	  for i, test_question in enumerate(test_questions):
#         # Replace commas in answers with empty strings (since csv is separated by comma)
#         # Answers in kaggle are processed in the same way
# 		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

# print(f"Completed! Result is in {result_file}")

## Testing

In [14]:
# newresult = []
# for res in result:
#     new_res = ''
#     fixed = False
#     if res[0] == '《':
#         closed = False
#         for i in range(1,len(res)):
#             if res[i] == '》':
#                 closed = True
#                 break
#         if not closed:
#             new_res = res + '》'
#             fixed = True
#             print(f"fixed: {new_res}")
        
    
#     elif res[-1] == '》':
#         closed = False
#         for i in range(len(res)-1, 0, -1):
#             if res[i] == '《':
#                 closed = True
#                 break
#         if not closed:
#             new_res = '《' + res
#             fixed = True
#             print(f"fixed: {new_res}")
#     if res[0] == '「':
#         closed = False
#         for i in range(1,len(res)):
#             if res[i] == '」':
#                 closed = True
#                 break
#         if not closed:
#             new_res = res + '」'
#             fixed = True
#             print(f"fixed: {new_res}")
    
#     elif res[-1] == '」':
#         closed = False
#         for i in range(len(res)-1, 0, -1):
#             if res[i] == '「':
#                 closed = True
#                 break
#         if not closed:
#             new_res = '「' + res
#             fixed = True
#             print(f"fixed: {new_res}")
    
#     if fixed:
#         newresult.append(new_res)
#     else:
#         newresult.append(res)

# newresult_file = "newresult_2.csv"
# with open(newresult_file, 'w') as f:	
# 	  f.write("ID,Answer\n")
# 	  for i, test_question in enumerate(test_questions):
#         # Replace commas in answers with empty strings (since csv is separated by comma)
#         # Answers in kaggle are processed in the same way
# 		    f.write(f"{test_question['id']},{newresult[i].replace(',','')}\n")


In [15]:
# p = test_paragraphs[test_questions[2]['paragraph_id']]
# print(p)
# comp = r"" + "置縣" + '\s*(?:\S\s*){0,20}' + "六年"
# print(re.search(comp,p))

In [16]:

# print("Evaluating Test Set ...")

# result = []

# model.eval()
# with torch.no_grad():
#     for i, data in enumerate(tqdm(test_loader)):
#         output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
#                        attention_mask=data[2].squeeze(dim=0).to(device))
#         result.append(evaluate(data, output))

# result_file = "result.csv"
# with open(result_file, 'w') as f:	
# 	  f.write("ID,Answer\n")
# 	  for i, test_question in enumerate(test_questions):
#         # Replace commas in answers with empty strings (since csv is separated by comma)
#         # Answers in kaggle are processed in the same way
# 		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

# print(f"Completed! Result is in {result_file}")